## This notebook contains the code implemented for my capstone project for the data analyst in SQL career track with Datacamp

In this project, I explored over 13000 best-selling video games created between 1977 and 2020 with sql. I compared a dataset on game sales with critic and user reviews to determine whether or not video games have improved as the gaming market has grown.

The goal of this project is to determine the best performing years for video games.

In [20]:
#importing all relevant libaries
import pandas as pd
import sqlite3 as sql

In [2]:
#reading the dataset
games = pd.read_csv('c:game_sales_data.csv',encoding= 'unicode_escape')
games_sales = games[['Name','Platform','Publisher','Developer', 'Total_Shipped','Year']]
games_critic = games[['Name', 'Critic_Score', 'User_Score']]

In [3]:
#creating and connecting games_sales_data database
database_connection = sql.connect('c:games_sales_data.db')

In [4]:
#creating a cursor 
c = database_connection.cursor()

In [5]:
# creating the sales table in the database
c.execute(
    '''
            CREATE TABLE sales (
                Name TEXT,
                Platform TEXT,
                Publisher TEXT,
                Developer TEXT,
                Total_Shipped REAL,
                Year INTEGER
                );
     ''')

In [6]:
# creating the critic table in the database
c.execute(
    '''
            CREATE TABLE critic (
                Name TEXT,
                Critic_Score REAL,
                User_Score REAL
                );
     ''')

In [7]:
#filling the tables with information from the datasets
games_sales.to_sql('sales', database_connection, if_exists = 'append', index = False)
games_critic.to_sql('critic', database_connection, if_exists = 'append', index = False)

19600

In [8]:
#viewing the sales table
pd.read_sql(
    '''
    SELECT *
    FROM sales
    LIMIT 10;
''', database_connection
)

,Name,Platform,Publisher,Developer,Total_Shipped,Year
0,Wii Sports,Wii,Nintendo,Nintendo EAD,82.90,2006
1,Super Mario Bros.,NES,Nintendo,Nintendo EAD,40.24,1985
2,Counter-Strike: Global Offensive,PC,Valve,Valve Corporation,40.00,2012
3,Mario Kart Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
4,PLAYERUNKNOWN'S BATTLEGROUNDS,PC,PUBG Corporation,PUBG Corporation,36.60,2017
5,Minecraft,PC,Mojang,Mojang AB,33.15,2010
6,Wii Sports Resort,Wii,Nintendo,Nintendo EAD,33.13,2009
7,Pokemon Red / Green / Blue Version,GB,Nintendo,Game Freak,31.38,1998
8,New Super Mario Bros.,DS,Nintendo,Nintendo EAD,30.80,2006
9,New Super Mario Bros. Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


In [9]:
#viewing the critic table
pd.read_sql(
    '''
    SELECT *
    FROM critic
    LIMIT 10;
''', database_connection
)

,Name,Critic_Score,User_Score
0,Wii Sports,7.7,8.0
1,Super Mario Bros.,10.0,8.2
2,Counter-Strike: Global Offensive,8.0,7.5
3,Mario Kart Wii,8.2,9.1
4,PLAYERUNKNOWN'S BATTLEGROUNDS,8.6,4.7
5,Minecraft,10.0,7.8
6,Wii Sports Resort,8.0,8.8
7,Pokemon Red / Green / Blue Version,9.4,8.8
8,New Super Mario Bros.,9.1,8.1
9,New Super Mario Bros. Wii,8.6,9.2


## What are the top ten best selling games of all time?

In [10]:
pd.read_sql(
    '''
    SELECT *
    FROM sales
    ORDER BY Total_Shipped DESC
    LIMIT 10;
''', database_connection
)

,Name,Platform,Publisher,Developer,Total_Shipped,Year
0,Wii Sports,Wii,Nintendo,Nintendo EAD,82.90,2006
1,Super Mario Bros.,NES,Nintendo,Nintendo EAD,40.24,1985
2,Counter-Strike: Global Offensive,PC,Valve,Valve Corporation,40.00,2012
3,Mario Kart Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
4,PLAYERUNKNOWN'S BATTLEGROUNDS,PC,PUBG Corporation,PUBG Corporation,36.60,2017
5,Minecraft,PC,Mojang,Mojang AB,33.15,2010
6,Wii Sports Resort,Wii,Nintendo,Nintendo EAD,33.13,2009
7,Pokemon Red / Green / Blue Version,GB,Nintendo,Game Freak,31.38,1998
8,New Super Mario Bros.,DS,Nintendo,Nintendo EAD,30.80,2006
9,New Super Mario Bros. Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


## What are the top ten best performing years in terms of number of video games sold?

In [11]:
pd.read_sql(
    '''
    SELECT year, SUM(total_shipped) AS total_games_sold
    FROM sales
    GROUP BY year
    ORDER BY total_games_sold DESC
    LIMIT 10;
''', database_connection
)

,Year,total_games_sold
0,2008,734.62
1,2009,694.39
2,2010,676.41
3,2007,634.62
4,2011,633.36
5,2006,520.71
6,2013,470.88
7,2005,456.29
8,2012,455.69
9,2014,454.16


The 2000's are better performing years in terms of number of copies of video games sold.
In 2008 over 734 million copies of video games were sold.

Although number of copies of video games sold is highly relevant in determining the best performing year for video games, it is almost important to consider the reviews from critics as well as number of games released.

## Which years released the most games?

In [12]:
pd.read_sql(
    '''
    SELECT s.year, COUNT(s.name) AS num_games
    FROM sales AS s
    INNER JOIN critic AS c
    ON s.name = c.name
    GROUP BY s.year
    ORDER BY num_games DESC
    LIMIT 10;
''', database_connection
)

,Year,num_games
0,2009,3450
1,2008,3270
2,2011,2902
3,2010,2859
4,2007,2779
5,2006,2203
6,2005,2181
7,2014,2148
8,2015,1953
9,2017,1810


The 2000's had high number of video games relases, this can be said to explain the higher number of copies sold.

## How many video games in the sales table do not have reviews in the critic table?

In [13]:
pd.read_sql(
    '''
    SELECT COUNT (s.name) AS 'Games without reviews'
    FROM sales AS s
    LEFT JOIN critic AS c
    ON s.name = c.name
    WHERE c.critic_score IS NULL AND c.user_score IS NULL;
''', database_connection
)

,Games without reviews
0,17248


In [14]:
pd.read_sql(
    '''
    SELECT *
    FROM sales AS s
    LEFT JOIN critic AS c
    ON s.name = c.name
    WHERE c.critic_score IS NULL AND c.user_score IS NULL;
''', database_connection
)

,Name,Platform,Publisher,Developer,Total_Shipped,Year,Name,Critic_Score,User_Score
0,Minecraft,PC,Mojang,Mojang AB,33.15,2010,Minecraft,None,None
1,Minecraft,PC,Mojang,Mojang AB,33.15,2010,Minecraft,None,None
2,Tetris,GB,Nintendo,Bullet Proof Software,30.26,1989,Tetris,None,None
3,Super Mario Land,GB,Nintendo,Nintendo R&D1,18.14,1989,Super Mario Land,None,None
4,Grand Theft Auto: San Andreas,PS2,Rockstar Games,Rockstar North,17.30,2004,Grand Theft Auto: San Andreas,None,None
...,...,...,...,...,...,...,...,...,...
17243,Donkey Kong,3DS,Nintendo,Nintendo,0.01,2011,Donkey Kong,None,None
17244,FirePower for Microsoft Combat Flight Simulator 3,PC,GMX Media,Shockwave Productions,0.01,2004,FirePower for Microsoft Combat Flight Simulator 3,None,None
17245,Ashita no Joe 2: The Anime Super Remix,PS2,Capcom,Capcom,0.01,2002,Ashita no Joe 2: The Anime Super Remix,None,None
17246,Tokyo Yamanote Boys for V: Main Disc,PSV,Rejet,Rejet,0.01,2017,Tokyo Yamanote Boys for V: Main Disc,None,None


17248 video games in the sales table have no review available in the critic table.

## Which years do the video games without reviews belong to?

In [15]:
pd.read_sql(
    '''
      SELECT DISTINCT year, COUNT(s.name) AS no_of_games
      FROM sales AS s
      LEFT JOIN critic AS c
      ON s.name = c.name
      WHERE c.critic_score IS NULL AND c.user_score IS NULL
      GROUP BY year
      ORDER BY no_of_games DESC
      LIMIT 10;
''', database_connection
)

,Year,no_of_games
0,2010,1508
1,2011,1418
2,2009,1414
3,2014,1300
4,2015,1197
5,2008,1100
6,2016,991
7,2007,865
8,2012,850
9,2017,837


The 2000's are the better performing years in terms of number of video games sold, however they also have a large number of games without reviews.

## What is the average critic score of videos games per year?

In [16]:
pd.read_sql(
   '''
    SELECT s.year, ROUND(AVG(c.Critic_Score),2) AS avg_critic_score
    FROM sales AS s
    INNER JOIN critic AS c
    ON s.name = c.name
    GROUP BY s.Year
    ORDER BY avg_critic_score DESC
    LIMIT 10;
''', database_connection
)

,Year,avg_critic_score
0,1984,9.50
1,1992,8.68
2,1990,8.54
3,1991,8.32
4,2020,8.26
5,1994,8.04
6,2019,7.97
7,1985,7.84
8,1993,7.72
9,2013,7.61


1984 had the highest average critic score.

## Comparing the number of games released and average critics score

In [17]:
pd.read_sql(
   '''
    SELECT s.year, COUNT(s.name) AS num_games, ROUND(AVG(c.critic_score),2) AS avg_critic_score
    FROM sales AS s
    INNER JOIN critic AS c
    ON s.name = c.name
    GROUP BY s.year
    HAVING COUNT(s.name) > 5
    ORDER BY avg_critic_score DESC
    LIMIT 10;
''', database_connection
)

,Year,num_games,avg_critic_score
0,1984,12,9.50
1,1992,56,8.68
2,1990,30,8.54
3,1991,39,8.32
4,2020,13,8.26
5,1994,131,8.04
6,2019,95,7.97
7,1985,19,7.84
8,1993,94,7.72
9,2013,1631,7.61


The number of video games released in a year can be said to have an impact on the year's average critic score.

It is important to also take into consideration the avearge user score given by the video games when deciding of the best performing years.

## Comparing the number of games released and average user scores

In [18]:
pd.read_sql(
   '''
    SELECT s.year, COUNT(s.name) AS num_games, ROUND(AVG(c.user_Score),2) AS avg_user_score
    FROM sales AS s
    INNER JOIN critic AS c
    ON s.name = c.name
    GROUP BY s.year
    HAVING COUNT(s.name) > 5
    ORDER BY avg_user_score DESC
    LIMIT 10;
''', database_connection
)

,Year,num_games,avg_user_score
0,1997,333,9.38
1,1990,30,9.30
2,1999,478,9.10
3,2003,1695,8.88
4,1998,423,8.86
5,1994,131,8.80
6,1987,18,8.80
7,2007,2779,8.64
8,2000,545,8.52
9,1996,351,8.40


Video games released in earlier years were ranked higher by users.

The best performing video games years will be determined based on three main categories:

1. Total number of copies of video games sold
2. Average critics scores
3. Average user scores

## Determining the top 20 best performing years for video games

In [19]:
pd.read_sql(
   '''
    SELECT s.year, SUM(total_shipped) AS total_games_sold, ROUND(AVG(c.critic_score),2) AS avg_critic_score, ROUND(AVG(c.user_Score),2) AS avg_user_score
    FROM sales AS s
    INNER JOIN critic AS c
    ON s.name = c.name 
    GROUP BY s.year
    HAVING ROUND(AVG(c.critic_score),2) >= 6.5 AND ROUND(AVG(c.user_Score),2) >= 6.5 AND COUNT(s.name) > 5
    ORDER BY total_games_sold DESC, avg_user_score DESC, avg_critic_score DESC
    LIMIT 20;
''', database_connection
)

,Year,total_games_sold,avg_critic_score,avg_user_score
0,2008,1954.78,6.60,8.33
1,2010,1887.95,7.05,8.07
2,2011,1758.02,7.08,6.68
3,2009,1694.93,6.73,8.13
4,2014,1649.74,7.58,6.86
5,2007,1617.75,6.69,8.64
6,2013,1600.93,7.61,6.78
7,2015,1263.99,7.52,6.79
8,2012,1209.84,7.45,6.97
9,2006,1111.91,6.91,8.01


The table shows that the top 20 performing years have high number of copies of video games sold, above average critics scores and average user scores